In [41]:
from models.mnist_classifier.lenet import LeNet5, LeNet1
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets.mnist import MNIST
from torchvision.datasets import FashionMNIST
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, TensorDataset, ConcatDataset
import random
import os
from  PIL import  Image
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torchvision import transforms,utils,datasets
import numpy as np
from torch import optim
import time
import torchvision.models as models

In [42]:
mnist_root = './autodl-tmp/mnist'
mnist_train = MNIST(mnist_root,
                   download=True,
                   transform=transforms.Compose([
                       transforms.Resize((32, 32)),
                       transforms.ToTensor(),
                       transforms.Normalize((0.5,), (0.5,))]))
mnist_test = MNIST(mnist_root,
                  train=False,
                  download=True,
                  transform=transforms.Compose([
                      transforms.Resize((32, 32)),
                      transforms.ToTensor(),
                      transforms.Normalize((0.5,), (0.5,))]))

In [43]:
class LoadData(Dataset):
    def __init__(self,txt_path,train_flag=True):
        self.tensors_info=self.get_tensors(txt_path)
        self.train_flag=train_flag

    def get_tensors(self, txt_path):
        with open(txt_path,'r',encoding='UTF-8') as f:
            tensors_info=f.readlines()
            tensors_info=list(map(lambda x:x.strip().split('\t'),tensors_info))
        return tensors_info

    def __getitem__(self, index):
        tensor_path,label=self.tensors_info[index]
        try:
            tensor = torch.load(tensor_path)
        except:
            print(tensor_path)
        #返回打开的图片和它的标签
        return tensor, int(label)

    def __len__(self):
        return len(self.tensors_info)

In [44]:
my_root = 'autodl-tmp/testcases/mnist/lenet5'

train_set=LoadData(my_root + "/train.txt",True)
test_set=LoadData(my_root + "/test.txt",False)

In [45]:
# 合并数据集
train_merged = ConcatDataset([mnist_train, train_set])
train_merged_loader = DataLoader(train_merged, batch_size=256, shuffle=True, num_workers=8)

test_merged = ConcatDataset([mnist_test, test_set])
test_merged_loader = DataLoader(test_merged, batch_size=256, num_workers=8)

In [46]:
net = LeNet5()
# net.load_state_dict(torch.load('autodl-tmp/save_model/mnist/Lenet1/lenet_epoch=2_test_acc=0.816.pth'))
net.train()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=1e-4)

In [47]:
def train(epoch):
    global cur_batch_win
    net.train()
    loss_list, batch_list = [], []
    for i, (images, labels) in enumerate(train_merged_loader):
        optimizer.zero_grad()
        output = net(images)
        loss = criterion(output, labels)
        loss_list.append(loss.detach().cpu().item())
        batch_list.append(i+1)
        if i % 10 == 0:
             print('Train - Epoch %d, Batch: %d, Loss: %f' % (epoch, i, loss.detach().cpu().item()))
        loss.backward()
        optimizer.step()

In [48]:
def test():
    net.eval()
    with torch.no_grad():
        total_correct = 0
        avg_loss = 0.0
        for i, (images, labels) in enumerate(test_merged_loader):
            output = net(images)
            avg_loss += criterion(output, labels).sum()
            pred = output.detach().max(1)[1]
            total_correct += pred.eq(labels.view_as(pred)).sum()

    avg_loss /= len(test_merged)  # 除以测试集大小
    acc = float(total_correct) / len(test_merged)
    print('Test Avg. Loss: %f, Accuracy: %f' % (avg_loss.detach().cpu().item(), acc))
    return acc

In [49]:
def train_and_test(epoch):
    print('training...')
    train(epoch)
    acc = test()
    return acc

In [50]:
for e in range(1, 20):
    acc = train_and_test(e)
    #if e % 2 == 0:
    #    torch.save(net.state_dict(), f'autodl-tmp/save_model/mnist/Lenet1/re_lenet_epoch={e}_test_acc={acc:0.3f}.pth')
    print(f'lenet_epoch={e}_test_acc={acc:0.3f}.pth')

training...
Train - Epoch 1, Batch: 0, Loss: 2.303067
Train - Epoch 1, Batch: 10, Loss: 2.295403
Train - Epoch 1, Batch: 20, Loss: 2.283597
Train - Epoch 1, Batch: 30, Loss: 2.271069
Train - Epoch 1, Batch: 40, Loss: 2.259309
Train - Epoch 1, Batch: 50, Loss: 2.225097
Train - Epoch 1, Batch: 60, Loss: 2.195620
Train - Epoch 1, Batch: 70, Loss: 2.136007
Train - Epoch 1, Batch: 80, Loss: 2.098171
Train - Epoch 1, Batch: 90, Loss: 2.052160
Train - Epoch 1, Batch: 100, Loss: 1.955610
Train - Epoch 1, Batch: 110, Loss: 1.804026
Train - Epoch 1, Batch: 120, Loss: 1.715901
Train - Epoch 1, Batch: 130, Loss: 1.634450
Train - Epoch 1, Batch: 140, Loss: 1.418393
Train - Epoch 1, Batch: 150, Loss: 1.282449
Train - Epoch 1, Batch: 160, Loss: 1.177955
Train - Epoch 1, Batch: 170, Loss: 1.026550
Train - Epoch 1, Batch: 180, Loss: 0.964814
Train - Epoch 1, Batch: 190, Loss: 0.880473
Train - Epoch 1, Batch: 200, Loss: 0.843273
Train - Epoch 1, Batch: 210, Loss: 0.808537
Train - Epoch 1, Batch: 220, Lo